In [1]:
from textblob import TextBlob
from IPython.display import Markdown, display
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
spark = SparkSession.builder.getOrCreate()

In [2]:
# Import twint
import sys
sys.path.append("twint/")
# Set up TWINT config
import twint
c = twint.Config()

In [3]:
data_path = '/home/dishant/spark-data'
file_path = data_path + "/bitcoin_tweets.csv"
df1 = spark.read.format("csv").option("header", "true").load(file_path)

In [4]:
df1.head(10)

[Row(UserName='Erin Malone', Handle='@ErinEMalone', Timestamp='2021-07-18T23:23:20.000Z', Text='1. 10 weeks ago, I joined the ', Emojis=None, Comments=None, Likes=None, Retweets=None),
 Row(UserName='@lightning', Handle=None, Timestamp=None, Text=None, Emojis=None, Comments=None, Likes=None, Retweets=None),
 Row(UserName=' network barely knowing how to open a channel. Last week I became a profitable lightning network routing node. ', Handle=None, Timestamp=None, Text=None, Emojis=None, Comments=None, Likes=None, Retweets=None),
 Row(UserName='Here\'s a thread on the ultimate strategy game for LN node operators from one #Bitcoin pleb to another."', Handle=None, Timestamp='76', Text='487', Emojis='1.9K', Comments=None, Likes=None, Retweets=None),
 Row(UserName='Linkamus ₿ ∞/21M', Handle='@VersusTheMan', Timestamp='2021-07-18T23:02:42.000Z', Text="I've bought as much #bitcoin as I can possibly afford in this 30k - 34k range. Dry powder is completely gone. Can you say the same?", Emojis=No

In [5]:
# Function to get sentiment 
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative

In [6]:
# Clean tweets
sentiment = udf(apply_blob, DoubleType())
df1.withColumn("sentiment", sentiment(df1['Text'])).show()

21/07/21 16:07:27 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)
java.io.IOException: Cannot run program "/usr/bin/python3.7": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:209)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:132)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:105)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:131)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:129)
	at org.apache.spark.rdd.RDD.$anonfun$map

Py4JJavaError: An error occurred while calling o49.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, 192.168.1.10, executor driver): java.io.IOException: Cannot run program "/usr/bin/python3.7": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:209)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:132)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:105)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:131)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evalu

In [21]:
# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())

In [22]:
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

In [26]:
# Transform tweets to pandas DF
df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])

TypeError: 'NoneType' object is not subscriptable

AttributeError: 'DataFrame' object has no attribute 'cols'

In [13]:
# Pretty printing the result
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [17]:
for tweet in df1:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("No result", color="grey")
        print("")

Column<b'UserName'>


TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'pyspark.sql.column.Column'>